<a href="https://colab.research.google.com/github/asokraju/Rienforcement-learning/blob/master/ddpg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu 

#!pip install tensorboardcolab
#from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

#tbc=TensorBoardColab()

     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 3.9MB 50.6MB/s 
     |████████████████████████████████| 450kB 56.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=179beda6999270d5a2ce7405b32afb4ada0e72125d0bc0ba00e7d37ada96b3d0
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc2 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.0
    Uninstal

In [0]:
import tensorflow as tf
import numpy as np
import gym
import argparse
from tensorflow import keras
from tensorflow.keras import Input, Model, Sequential, layers
print(tf.__version__)
tf.test.gpu_device_name()

2.1.0


'/device:GPU:0'

In [0]:
#===================================
#     Actor and Critic DNNs
#===================================


class ActorNetwork(object):
  def __init__(self, state_dim, action_dim, action_bound, learning_rate, tau, batch_size):
    self.state_dim = state_dim
    self.action_dim = action_dim
    self.action_bound = action_bound
    self.learning_rate = learning_rate
    self.tau  = tau
    self.batch_size = batch_size
    self.optimizer = tf.keras.optimizers.Adam(self.learning_rate)

    #actor network
    self.inputs, self.out, self.scaled_out = self.create_actor_network()
    self.actor_model = keras.Model(inputs=self.inputs, outputs=self.scaled_out, name='actor_network')
    self.network_params = self.actor_model.trainable_variables

    #target actor network
    self.target_inputs, self.target_out, self.target_scaled_out = self.create_actor_network()
    self.target_actor_model = keras.Model(inputs=self.target_inputs, outputs=self.target_scaled_out, name='target_actor_network')
    self.target_network_params = self.target_actor_model.trainable_variables


  def create_actor_network(self):
    inputs = Input(shape = (self.state_dim,), batch_size = None, name = "actor_input_state")
    net = layers.Dense(400, name = 'actor_dense_1')(inputs)
    net = layers.BatchNormalization()(net)
    net = layers.Activation(activation=tf.nn.relu)(net)

    net = layers.Dense(300, name = 'actor_dense_2')(net)
    net = layers.BatchNormalization()(net)
    net = layers.Activation(activation=tf.nn.relu)(net)
    
    w_init = tf.random_uniform_initializer(minval=-0.03, maxval=0.03, seed=None)
    out = layers.Dense(self.action_dim, activation='tanh', name = 'actor_dense_3', kernel_initializer = w_init)(net)
    scaled_out = tf.multiply(out, self.action_bound, name = "actions_scaling")
    return inputs, out, scaled_out
  
  def update_target_network(self):
    self.update_target_network_params = [self.target_network_params[i].assign(tf.multiply(self.network_params[i], self.tau) + tf.multiply(self.target_network_params[i], 1-self.tau)) for i in range(len(self.target_network_params))]
  
  def train(self, inputs, a_gradient):
    with tf.GradientTape() as self.tape:
      self.prediction = self.actor_model(inputs)
    self.unnormalized_actor_gradients = self.tape.gradient(self.prediction, self.network_params, output_gradients = -a_gradient)
    self.actor_gradients = list(map(lambda x: tf.math.divide(x, self.batch_size), self.unnormalized_actor_gradients))
    self.optimizer.apply_gradients(zip(self.actor_gradients, self.network_params))
    
  def predict(self, inputs):
    return self.actor_model(inputs)

  def predict_target(self, inputs):
    return self.target_actor_model(inputs)


In [0]:
class CriticNetwork(object):
  def __init__(self, state_dim, action_dim, action_bound, learning_rate, tau, gamma):
    self.state_dim = state_dim
    self.action_dim = action_dim
    self.action_bound = action_bound
    self.learning_rate = learning_rate
    self.tau  = tau
    self.gamma = gamma
    self.optimizer = tf.keras.optimizers.Adam(self.learning_rate)

    #Critic Network and parameters
    self.inputs_state, self.inputs_action, self.out = self.create_critic_network()
    self.critic_model = keras.Model(inputs=[self.inputs_state, self.inputs_action], outputs=self.out, name='critic_network')
    self.network_params = self.critic_model.trainable_variables

    #Target Critic Network and parameters
    self.target_inputs_state, self.target_inputs_action, self.target_out = self.create_critic_network()
    self.target_critic_model = keras.Model(inputs=[self.target_inputs_state, self.target_inputs_action], outputs=self.target_out, name='target_critic_network')
    self.target_network_params = self.target_critic_model.trainable_variables

    #gradients of Q function with respect to actions
    
  def create_critic_network(self):
    inputs_state = Input(shape = (self.state_dim,), batch_size = None, name = "critic_input_state")
    inputs_action = Input(shape = (self.action_dim,), batch_size = None, name = "critic_input_action")
    
    #first hidden layer
    net_state = layers.Dense(400, name = 'critic_dense_1')(inputs_state)
    net_state = layers.BatchNormalization()(net_state)
    net_state = layers.Activation(activation=tf.nn.relu)(net_state)

    # second hidden layer
    net_state = layers.Dense(300, name = 'critic_dense_2_state')(net_state)
    net_action = layers.Dense(300, name = 'critic_dense_2_action')(inputs_action)
    net = layers.Add()([net_state, net_action])
    net = layers.BatchNormalization()(net)
    net = layers.Activation(activation=tf.nn.relu)(net)

    w_init = tf.random_uniform_initializer(minval=-0.03, maxval=0.03, seed=None)
    out = layers.Dense(1, name = 'Q_val', kernel_initializer = w_init)(net)
    return inputs_state, inputs_action, out

  def update_target_network(self):
    self.update_target_network_params = [self.target_network_params[i].assign(tf.multiply(self.network_params[i], self.tau) + tf.multiply(self.target_network_params[i], 1-self.tau)) for i in range(len(self.target_network_params))]

  def train(self, input_state, input_actions, predicted_q_val):
    with tf.GradientTape() as self.tape:
      self.prediction = self.critic_model([input_state, input_actions])
      self.loss = tf.keras.losses.MSE(self.prediction, predicted_q_val)
    self.gradients = self.tape.gradient(self.loss, self.network_params)
    self.optimizer.apply_gradients(zip(self.gradients, self.network_params))
    return self.critic_model([input_state, input_actions])
  
  def action_gradient(self, input_state, input_actions):
    var = tf.constant(input_actions)
    with tf.GradientTape(watch_accessed_variables=False) as self.tape:
      self.tape.watch(var)
      self.prediction = self.critic_model([input_state, var])
    return self.tape.gradient(self.prediction, var)
  
  def predict(self, inputs_state, inputs_actions):
    return self.critic_model([inputs_state, inputs_actions])
  
  def predict_target(self, inputs_state, inputs_actions):
    return self.target_critic_model([inputs_state, inputs_actions])


In [0]:
# Taken from https://github.com/openai/baselines/blob/master/baselines/ddpg/noise.py, which is
# based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckActionNoise:
    def __init__(self, mu, sigma=0.3, theta=.15, dt=1e-2, x0=None):
        self.theta = theta
        self.mu = mu
        self.sigma = sigma
        self.dt = dt
        self.x0 = x0
        self.reset()

    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + \
                self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
        self.x_prev = x
        return x

    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)

    def __repr__(self):
        return 'OrnsteinUhlenbeckActionNoise(mu={}, sigma={})'.format(self.mu, self.sigma)

In [0]:
# Replay buffer
# Taken from https://github.com/pemami4911/deep-rl/blob/master/ddpg/replay_buffer.py
from collections import deque
import random
import numpy as np

class ReplayBuffer(object):

    def __init__(self, buffer_size, random_seed=123):
        """
        The right side of the deque contains the most recent experiences 
        """
        self.buffer_size = buffer_size
        self.count = 0
        self.buffer = deque()
        random.seed(random_seed)

    def add(self, s, a, r, t, s2):
        experience = (s, a, r, t, s2)
        if self.count < self.buffer_size: 
            self.buffer.append(experience)
            self.count += 1
        else:
            self.buffer.popleft()
            self.buffer.append(experience)

    def size(self):
        return self.count

    def sample_batch(self, batch_size):
        batch = []

        if self.count < batch_size:
            batch = random.sample(self.buffer, self.count)
        else:
            batch = random.sample(self.buffer, batch_size)

        s_batch = np.array([_[0] for _ in batch])
        a_batch = np.array([_[1] for _ in batch])
        r_batch = np.array([_[2] for _ in batch])
        t_batch = np.array([_[3] for _ in batch])
        s2_batch = np.array([_[4] for _ in batch])

        return s_batch, a_batch, r_batch, t_batch, s2_batch

    def clear(self):
        self.buffer.clear()
        self.count = 0




In [0]:
args = {
    'summary_dir' : '/content/sample_data',
    'buffer_size' : 1000000,
    'random_seed' : 1754,
    'max_episodes': 600,
    'max_episode_len' : 200,
    'mini_batch_size': 64
}

In [0]:
def train(env, args, actor, critic, actor_noise, reward_result):
  writer = tf.summary.create_file_writer(logdir = args['summary_dir'])
  actor.update_target_network()
  critic.update_target_network()
  replay_buffer = ReplayBuffer(int(args['buffer_size']), int(args['random_seed']))

  paths = list()

  for i in range(args['max_episode_len']):
    s = env.reset()
    ep_reward = 0
    ep_ave_max_q = 0
    obs, actions, rewards = [], [], []
    for j in range(args['max_episode_len']):
      a = actor.predict(np.reshape(s,(1,actor.state_dim))) + actor_noise()
      #print(j,a)
      s2, r, terminal, info = env.step(a[0])
      replay_buffer.add(np.reshape(s, (actor.state_dim,)), np.reshape(a, (actor.action_dim,)), r, terminal, np.reshape(s2, (actor.state_dim,)))
      if replay_buffer.size() > args['mini_batch_size']:
        s_batch, a_batch, r_batch, t_batch, s2_batch = replay_buffer.sample_batch(args['mini_batch_size'])
        target_q = np.array(critic.predict_target(s2_batch, actor.predict_target(s2_batch)))
        #print("target_q", i,j, target_q[0])
        y = []
        for k in range(args['mini_batch_size']):
          if t_batch[k]:
            y.append(r_batch[k])
          else:
            y.append(r_batch[k] + critic.gamma * np.array(target_q[k]))
        #print(i,j,np.shape(s_batch))
        #print(i,j,np.shape(a_batch))
        #print(i,j,np.shape(y))
        temp = np.reshape(y, (args['mini_batch_size'],1))
        #print("temp",i,j,temp[0])
        predicted_q_value = critic.train(s_batch, a_batch, temp.astype('float32'))
        #print("predicted_q_value",i,j,predicted_q_value[0])
        #print(np.amax(predicted_q_value))
        ep_ave_max_q += np.amax(predicted_q_value)
        #print("ep_ave_max_q",i,j,ep_ave_max_q)

        a_outs = actor.predict(s_batch)
        #print("a_outs",i,j,a_outs[0])
        grads = critic.action_gradient(s_batch, a_outs)
        #print("grads",i,j,grads[0])
        actor.train(s_batch, grads)
        actor.update_target_network()
        critic.update_target_network()

      s = s2
      ep_reward += r
      obs.append(s)
      actions.append(a[0])
      rewards.append(r)

      if terminal:
        with writer.as_default():
          tf.summary.scalar("Reward", ep_reward, step = i)
          tf.summary.scalar("Qmax Value", ep_ave_max_q / float(j), step = i)
          writer.flush()
        print('| Reward: {:.4f} | Episode: {:d} | Qmax: {:.4f}'.format((ep_reward), i, (ep_ave_max_q / float(j))))
        reward_result[i] = ep_reward
        path = {
            "Observation":np.concatenate(obs).reshape((200,3)), 
            "Action":np.concatenate(action), 
            "Reward":np.asarray(rewards)
            }
        paths.append(path)
        break






In [0]:
tf.keras.backend.set_floatx('float32')
env = gym.make('Pendulum-v0')
state = env.reset()
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_bound = env.action_space.high

actor = ActorNetwork(state_dim = state_dim, action_dim=action_dim, action_bound=action_bound, learning_rate=0.0001, tau=0.001, batch_size=args['mini_batch_size'])
#inputs, out, scaled_out = actor.create_actor_network()
#actor_model = keras.Model(inputs=inputs, outputs=scaled_out, name='actor_network')
#print(actor.actor_model.summary())
#actor.update_target_network()
action = actor.actor_model(np.reshape(state, (1,actor.state_dim)))
#rint(state, action)

critic = CriticNetwork(state_dim = state_dim, action_dim=action_dim, action_bound=action_bound, learning_rate=0.001, tau=0.001, gamma=.99)
#inputs_state, inputs_action, out = critic.create_critic_network()
#critic_model = keras.Model(inputs=[inputs_state, inputs_action], outputs=out, name='critic_network')
#print(critic.critic_model.summary())
#critic.update_target_network()
#keras.utils.plot_model(critic.critic_model, 'critic_model.png')
#keras.utils.plot_model(actor.actor_model, 'actor_model.png')
actor_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(action_dim))
reward_result = np.zeros(2500)
train(env, args, actor, critic, actor_noise, reward_result)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended

In [0]:
state = np.reshape(env.reset(), (1,actor.state_dim))
action = np.array(actor.actor_model(state))
print(state, action)
q_val = np.asarray(critic.critic_model([state,action]))
print(q_val)
critic.train(state, action, q_val+100)
q_val = np.asarray(critic.critic_model([state,action]))
print(q_val)
state_batch = np.reshape(np.asarray([state, state]),(2,actor.state_dim))
action_batch = np.reshape(np.asarray([action, action]),(2,actor.action_dim))
grads = critic.action_gradient(state_batch, action_batch)
print("gradient of Q wrt actions: {}".format(grads))
actor.train(state_batch, grads)
critic.predict_target(state_batch, action_batch)
temp = critic.predict(state_batch, action_batch)
temp[1]
np.reshape(critic.critic_model([state_batch, action_batch]), (len(state_batch),1))

In [0]:
q_val = np.asarray(critic.target_critic_model([state_batch,action_batch]))
q_val.dtype

In [0]:
#!pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()